# Сессия 3<br>
В данной сессии нам предстоит обучить наши выбранные модели на тестовых данных и проверить точность. Далее настроить подбор гиперпараметров и выбрать наилучший результат

In [1]:
# импорты 
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import warnings
warnings.filterwarnings("ignore")

### Загрузка данных

In [2]:
df = pd.read_csv("fullData.csv", index_col=0)

In [3]:
oneTar = pd.read_csv("OneTAR.csv", index_col=0)

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.ensemble import RandomForestClassifier

#### Подготовка данных к обучению, разделение на тестовую и тренировочную выборки

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
dr=[]
for i in range(oneTar.shape[0]):
    if "Юниоры" in oneTar["T"][i] or "юниоры" in oneTar["T"][i]:
        dr.append(i)

In [7]:
oneTar.drop(dr,axis=0,inplace=True)

In [8]:
oneTar.reset_index(drop=True, inplace=True)

In [9]:
dr=[]
for i in range(df.shape[0]):
    if "Юниоры" in df["Компетенция"][i] or "юниоры" in df["Компетенция"][i]:
        dr.append(i)

In [10]:
df.drop(dr,axis=0,inplace=True)

In [11]:
df.reset_index(drop=True, inplace=True)

In [12]:
cv = CountVectorizer()
tf = TfidfTransformer()

In [13]:
Xtr = oneTar["W"]
ytr = oneTar["T"]

In [14]:
Xtrcv = cv.fit_transform(Xtr)
Xtrtf = tf.fit_transform(Xtrcv)

In [15]:
df.dropna(subset=["Чистые слова"], inplace=True)

In [16]:
Xtest = df["Чистые слова"]
ytest = df["Компетенция"]

In [17]:
models = [LogisticRegression(),
          KNeighborsClassifier(n_neighbors=5), 
          RandomForestClassifier(random_state=42, n_estimators=100), 
          ]

#### Мы используем три модели без каких либо параметров, как видно ниже все модели справились примерно одинакого и не очень хорошо. Тем не менее лучше всего справился метод случайных соседей с точностью 20%

In [18]:
TestModels = pd.DataFrame()
temp = {}
for model in models:
    m = str(model)
    print(m)
    temp['Model'] = m[:m.index('(')]
    
    text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', model),
    ])
    
    text_clf = text_clf.fit(Xtr, ytr)
    pred = text_clf.predict(Xtest)
    
    temp['accuracy_score'] = accuracy_score(pred, ytest)
    print('accuracy_score', accuracy_score(pred, ytest))
    TestModels = TestModels.append([temp])
TestModels.set_index('Model', inplace=True)

LogisticRegression()
accuracy_score 0.07610146862483311
KNeighborsClassifier()
accuracy_score 0.19893190921228304
RandomForestClassifier(random_state=42)
accuracy_score 0.1054739652870494


### Проведем подбор гиперпараметров, это должно улучшить точность моделей

In [19]:
LR = LogisticRegression()
KNC = KNeighborsClassifier()
RFC = RandomForestClassifier()

In [20]:
# Параметры для LogisticRegression()
C = [10,25,50,100,150]
solver = ['newton-cg', 'sag', 'saga', 'lbfgs']

# Параметры для RandomForestClassifier()
n_estimators = [50,100,200,300,500]
max_features = [.5,.7]
max_depth = [3,6]

# Параметры для KNeighborsClassifier()
n_neighbors=[5,10,15,20]
p=[1,2]

### Ниже функция которая считает точность переданной ей модели 

In [21]:
def paramsearch(model):
    text_clf = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', model),
    ])
    text_clf = text_clf.fit(Xtr, ytr)
    pred = text_clf.predict(Xtest)
    # print('accuracy_score', accuracy_score(pred, ytest))
    return accuracy_score(pred, ytest)

#### Сделаем полный перебор по всем параметрам для каждой модели и запишем это в датафрейм

In [22]:
lrdf = pd.DataFrame()
tmp = {}
for i in C:
    for j in solver:
        tmp["C"] = i
        tmp["solver"] = j
        LR.C = i
        LR.solver = j
        ac = paramsearch(LR)
        tmp["acc"] = ac
        lrdf = lrdf.append([tmp])
        print("d", end=" ")

d d d d d d d d d d d d d d d d d d d d 

In [23]:
kncdf = pd.DataFrame()
tmp = {}
for i in n_neighbors:
    for j in p:
        tmp["n_neighbors"] = i
        tmp["p"] = j
        KNC.n_neighbors = i
        KNC.p = j
        ac = paramsearch(KNC)
        tmp["acc"] = ac
        kncdf = kncdf.append([tmp])
        print("d", end=" ")

d d d d d d d d 

In [24]:
rfdf = pd.DataFrame()
tmp = {}
for i in n_estimators:
    for j in max_features:
        for x in max_depth:
            tmp["n_estimators"] = i
            tmp["max_features"] = j
            tmp["max_depth"] = x
            RFC.n_estimators = i
            RFC.max_features = j
            RFC.max_depth = x
            ac = paramsearch(RFC)
            tmp["acc"] = ac
            rfdf = rfdf.append([tmp])
            print("d", end=" ")

d d d d d d d d d d d d d d d d d d d d 

#### Найдем в каждом датафрейме значение с максимальной точностью это и будут наши гиперпараметры 

In [25]:
lrdf[lrdf["acc"] == lrdf["acc"].max()]

,C,solver,acc
0,25,sag,0.742323


In [26]:
kncdf[kncdf["acc"] == kncdf["acc"].max()]

,n_neighbors,p,acc
0,5,2,0.198932


In [27]:
rfdf[rfdf["acc"] == rfdf["acc"].max()]

,n_estimators,max_features,max_depth,acc
0,500,0.5,6,0.339119


#### Как видно лучший результат дает логистическая регрессия с точностью в 74% с гиперпараметрами `{"C": 25, "solver": "saga"}`

In [28]:
lr = LogisticRegression(C=25, solver="saga")
text_clf = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', lr),
    ])
text_clf = text_clf.fit(Xtr, ytr)


In [29]:
import pickle

In [30]:
with open("model.pkl", 'wb') as file:
    pickle.dump(text_clf, file)


In [31]:
w = df["Чистые слова"][4]

In [38]:
pred = text_clf.predict([w])


In [39]:
pred

array(['Медицинская оптика'], dtype=object)